In [6]:
import cv2
import numpy as np

In [7]:
net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')
with open('coco.names','r')as f:
    classes = [line.strip() for line in f.readlines()]
colors = np.random.uniform(0,255,size=(len(classes),3))
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
print(output_layers)

['yolo_82', 'yolo_94', 'yolo_106']


In [8]:
def yolo_function(img):

    height,width,channel = img.shape
    blob = cv2.dnn.blobFromImage(img,0.00392,(416,416),(0,0,0),swapRB=True,crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)
                x = int(center_x -w/2)
                y = int(center_y - h/2)
                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    img2 = img.copy()
    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
    for i in range(len(boxes)):
        if i in indexes:
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            print(x,y,w,h)
            cv2.rectangle(img2,(x,y),(x+w,y+h),color,3)
            cv2.putText(img2,label,(x,y-20),cv2.FONT_HERSHEY_COMPLEX,1,color,2)
    return img2
    

In [10]:
video = cv2.VideoCapture('Cars Moving On Road Stock Footage - Free Download.mp4')
while video.isOpened():
    _, frame = video.read()
    out = yolo_function(frame)
    cv2.imshow('video', out)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()


46 -2 84 27
153 -2 95 26
614 2 33 18
677 2 36 17
730 0 31 8
1229 33 39 57
154 0 96 23
616 1 32 18
677 2 37 17
730 0 33 8
1229 34 38 56
160 0 90 22
679 2 36 18
731 0 33 9
617 6 31 18
1229 35 37 55
164 0 82 22
732 0 32 9
615 6 35 19
680 5 36 19
1228 36 38 54
167 -1 78 24
733 1 31 10
614 6 35 21
678 5 37 21
1228 38 37 52
173 1 63 18
734 0 31 12
615 8 34 22
679 6 36 21
1228 38 37 52
179 1 59 18
735 1 31 12
613 7 36 24
679 7 36 21
1229 38 35 52
736 0 32 13
613 7 36 24
678 10 39 21
1230 39 34 51
187 0 56 20
737 1 33 14
613 7 36 26
677 11 42 22
1231 39 32 51
189 0 54 20
739 1 33 16
612 7 36 26
677 10 43 23
1231 40 31 50
664 2 29 10
736 1 39 16
610 7 35 27
679 9 43 27
1232 40 30 50
664 1 32 11
738 1 39 18
680 9 43 29
608 20 40 21
1231 39 31 50
664 1 33 12
739 1 39 18
604 17 45 26
680 15 43 27
1231 37 31 51
665 0 33 13
677 2 35 14
748 1 36 17
603 11 44 35
681 18 41 25
1232 37 31 51
665 1 33 14
748 2 38 17
602 12 43 36
681 20 44 26
1232 36 32 53
618 0 26 11
666 0 32 15
742 1 48 27
602 14 44 37
6

696 178 106 113
1107 397 173 334
618 0 27 7
685 2 38 14
763 0 43 33
1244 22 29 55
689 49 52 53
805 44 68 65
691 187 116 130
616 0 28 9
685 2 40 16
762 1 47 36
1245 23 29 54
689 54 52 51
812 49 66 63
810 50 71 70
690 212 128 132
615 -1 29 10
682 3 41 16
766 2 46 38
1245 23 29 54
686 59 58 58
815 55 74 72
706 224 120 140
615 0 30 10
685 2 39 17
767 5 45 38
1246 22 28 55
688 61 54 60
818 59 76 74
704 256 141 153
615 0 29 11
687 4 39 22
771 9 42 37
1246 23 28 54
683 68 65 61
822 63 80 77
708 277 150 170
616 0 29 11
686 5 43 22
769 8 49 41
1246 23 28 54
687 73 64 61
824 64 86 85
715 296 158 185
617 0 28 11
686 2 36 13
686 7 44 22
772 10 50 42
1247 24 28 53
687 81 69 62
830 79 100 82
714 315 171 216
617 0 29 13
686 8 46 24
775 12 47 44
1247 24 28 53
688 88 72 61
840 88 96 81
721 364 187 229
615 1 31 14
664 1 23 8
686 7 46 26
776 14 50 48
1247 24 28 53
693 95 67 63
845 92 100 86
739 403 207 266
614 1 33 15
685 1 46 33
778 18 52 47
1247 23 28 54
692 100 72 67
689 108 85 65
849 100 107 90
734 4